In [1]:
import os


In [2]:
%pwd


'e:\\chichken_disease_detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\chichken_disease_detection'

In [ ]:
from dataclasses import dataclass
from pathlib import Path   

In [ ]:
#config entity

from dataclasses import dataclass
from pathlib import Path   

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [42]:
from chicken_disease_prediction.constants import *
from chicken_disease_prediction.utils.common import read_yaml, create_directories

In [59]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH ,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config= self.config.data_ingestion

        create_directories([config.root_dir])
        data_ingestion_config =DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )
        return data_ingestion_config
 

In [60]:
import os
import urllib.request as request
import zipfile
from chicken_disease_prediction import logger
from chicken_disease_prediction.utils.common import get_size


In [61]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config= config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        '''
        zip_file_path: src
        Extracts the zip file into the data directory 
        function returns none
        '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [62]:
import traceback

try:
    config = ConfigurationManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    print("An error occurred:", str(e))
    traceback.print_exc()
    raise

[2025-05-09 11:46:43,556: INFO: common: yaml file: config\config.yaml loaded Successfully]
[2025-05-09 11:46:43,559: INFO: common: yaml file: params.yaml loaded Successfully]
[2025-05-09 11:46:43,562: INFO: common: created directory at artifacts]
[2025-05-09 11:46:43,566: INFO: common: created directory at artifacts/data_ingestion]
[2025-05-09 11:47:01,966: INFO: 3673814057: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 24191732
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "3a1a6043dcc8424a06a8c9065c2a8f60a8fe1da2aa24afac43a6f20fe0acbf16"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E533:2795AC:759563:910D15:681DA4D7
Accept-Ranges: bytes
Date: Fri, 09 May 2025 06:46:51 GMT
Via: 1.1 varnish
X-Served-By: cache-fjr990033-FJR
X-Cache: M